## Import DOCX file

In [3]:
from docx import Document

In [8]:
document = Document(r"../res/test-swiss.docx")
document

In [15]:
for table in document.tables:
    for row in table.rows:
        for cell in row.cells:
            print(cell.text)

COUNTRY: SWITZERLAND
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social interactions

Retail
Upon RS
YES       
YES
Social Contact 

Social interactions

Professionals

Pro-actively
YES
YES
Social Contact 

Social interactions

Professionals

Upon RS
YES
YES
Social Contact 

Provide contact details
Retail
Pro-actively 
YES
YES
Social Contact 

Provide contact details
Retail
Upon RS 
YES
YES
Social Contact 

Provide contact details
Professionals

Pro-actively 
YES
YES
Social Contact 

Provide contact details
Professionals

Upon RS 
YES
YES
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social inte

Credit card agreements including related forms
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Professionals
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Professionals
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Provi

Professionals
Upon RS
YES
NO
Preparatory administrative activities 

Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Retail
Upon RS
YES
NO
Preparatory administra

Performance of activities 

Sector research and financial research 
with advisory mandate
with RSSI 

Retail
Upon RS 
YES
NO
Performance of activities 

Sector research and financial research 
with advisory mandate
with RSSI 

Professionals
Pro-actively 
YES
NO 
Performance of activities 

Sector research and financial research 
with advisory mandate
with RSSI 

Professionals
Upon RS 
YES
NO 
Performance of activities 

Sector research and financial research 
without advisory mandate
Retail
Pro-actively 
NO 
NO 
Performance of activities 

Sector research and financial research 
without advisory mandate
Retail
Upon RS 
YES
NO
Performance of activities 

Sector research and financial research 
without advisory mandate
Professionals
Pro-actively 
NO 
NO 
Performance of activities 

Sector research and financial research 
without advisory mandate
Professionals
Upon RS 
YES
YES
Performance of activities 

Macroeconomic research
with advisory mandate
with RSSI 

Retail
Pro-actively 
YES 
YE

## Make dataframes from tables

In [17]:
import pandas as pd
import io
import csv
from docx import Document

def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise

In [18]:
dfs = read_docx_tables(r"../res/test-swiss.docx")

In [19]:
dfs

[Empty DataFrame
 Columns: [COUNTRY: SWITZERLAND]
 Index: [],
            PROSPECTS              PROSPECTS.1      PROSPECTS.2  \
 0             GROUP                  ACTIVITY      CLIENT TYPE   
 1  Social Contact \n    Social interactions\n           Retail   
 2  Social Contact \n    Social interactions\n           Retail   
 3  Social Contact \n    Social interactions\n  Professionals\n   
 4  Social Contact \n    Social interactions\n  Professionals\n   
 5  Social Contact \n  Provide contact details           Retail   
 6  Social Contact \n  Provide contact details           Retail   
 7  Social Contact \n  Provide contact details  Professionals\n   
 8  Social Contact \n  Provide contact details  Professionals\n   
 
         PROSPECTS.3        PROSPECTS.4       PROSPECTS.5  
 0  INTERACTION MODE  OFFSHORE SCENARIO  ONSHORE SCENARIO  
 1      Pro-actively                YES               YES  
 2           Upon RS         YES                      YES  
 3      Pro-actively      

In [22]:
dfs[0]

,COUNTRY: SWITZERLAND


In [23]:
dfs[1]

,PROSPECTS,PROSPECTS.1,PROSPECTS.2,PROSPECTS.3,PROSPECTS.4,PROSPECTS.5
0,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,OFFSHORE SCENARIO,ONSHORE SCENARIO
1,Social Contact \n,Social interactions\n,Retail,Pro-actively,YES,YES
2,Social Contact \n,Social interactions\n,Retail,Upon RS,YES,YES
3,Social Contact \n,Social interactions\n,Professionals\n,Pro-actively,YES,YES
4,Social Contact \n,Social interactions\n,Professionals\n,Upon RS,YES,YES
5,Social Contact \n,Provide contact details,Retail,Pro-actively,YES,YES
6,Social Contact \n,Provide contact details,Retail,Upon RS,YES,YES
7,Social Contact \n,Provide contact details,Professionals\n,Pro-actively,YES,YES
8,Social Contact \n,Provide contact details,Professionals\n,Upon RS,YES,YES


## Assemble in app